In [1]:
import numpy as np
np.random.seed(1)

In [2]:

features=0
classes=0
samples=0
def modify_train_y(classes,train_y):
    new_train_y = []
    for x in train_y:
        current_y = []
        for i in range(classes):
            current_y.append(0)
        current_y[int(x)-1]=1
        new_train_y.append(current_y)
    return np.array(new_train_y).T


def data_loader(filename,isTrainData):
    # open data file
    file = open('./Dataset/'+filename,"r")


    # initialize
    i=0
    global features
    global classes
    global samples


    listx = []
    listy = []

    for line in file:

        fields = line.split()
        templist = []
        features = len(fields)-1
        for j in range(features):
                #print(fields[j])
            templist.append(float(fields[j]))

        listx.append(templist)
        listy.append(int(fields[features]))


    if isTrainData:
        samples = len(listx)
        classes = len(set(listy))
        
    # convert into numpy array
    x = np.array(listx)
    y = np.array(listy)
    
    #print(x.shape, y.shape)
    
    return x,y


In [3]:
train_x,train_y =  data_loader("trainNN.txt",1)
train_x = train_x.T
train_y  = modify_train_y(classes,train_y)
print("Features: ",str(features), " Classes: ",str(classes)," Samples: ",str(samples))

Features:  4  Classes:  4  Samples:  500


In [4]:
test_x,test_y =  data_loader("testNN.txt",0)
test_x = test_x.T

In [5]:
def sigmoid(z):
    return (1.0/(1.0+np.exp(-z)))
def derivative_sigmoid(z):
    return sigmoid(z)*(1-sigmoid(z))
def error(target,output):
    return  np.sum((target-output)*(target-output))/2.0
def derivative_error(target,output):
    return target-output



In [6]:
class Layer:
    def __init__(self,in_nodes,out_nodes):
        self.weight = np.random.randn(out_nodes,in_nodes)
        max_value = np.max(self.weight)
        self.weight/=max_value

        self.bias = np.random.randn(out_nodes,1)
        max_value = np.max(self.bias)
        self.bias/=max_value

        self.Z = None
        self.A = None

        self.dZ = None
        self.dA = None

        self.prev_A = None

    def forward(self,X):
        self.prev_A = X

        self.Z = np.dot(self.weight,X)+self.bias
        self.A = sigmoid(self.Z)

        return self.A


    def backward(self,delta,isLast , m ,learning_rate):

        if isLast:
            self.dA = self.A-delta
        else:
            self.dA = delta

        self.dZ = self.dA * derivative_sigmoid(self.A)

        self.dW = np.dot(self.dZ, self.prev_A.T)*(1.0/m)
        self.db =  np.sum(self.dZ, axis=1, keepdims=True)*(1.0/m)

        self.weight = self.weight  - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db

        return np.dot(self.weight.T, delta)





In [7]:
class Network:
    def __init__(self,features, classes, n_layers, size_layers):
        self.layers = []

        for i in range(1,len( size_layers)):
            self.layers.append(Layer(size_layers[i-1],size_layers[i]))
            # print(self.layers[-1].weight.shape)
        
    def train(self,X,Y,epoch =1000, learning_rate = 0.1):
        for i in range(epoch):
            A = X
            for layer in self.layers:
                A = layer.forward(A)
            
            delta = Y
            for layer in reversed(self.layers):
                delta = layer.backward(delta, layer == self.layers[-1] ,Y.shape[1],learning_rate)   

            if((i+1)%100==0):
                print('Iteration: ',str(i+1),' Error: ',error(self.layers[-1].A,Y))

        print('Train done!')

    def decide(self, X):
        A = X
        for layer in self.layers:
            Z = np.dot(layer.weight,A)+layer.bias
            A = sigmoid(Z)

        y_temp = A.T.tolist() 
        #print(y_temp)
        y_hat  = []
        for row in y_temp:
            y_hat.append(row.index(max(row)) + 1)

        return y_hat

    def test(self,X,Y):
        m = len(Y)

        y_hat = self.decide(X)

        n_miss = 0.0
        for i in range(m):
            #print("Expected : %d    Found : %d" %(Y[i], y_hat[i]))
            print('Expected: ',Y[i],' Found: ',y_hat[i])
            if (Y[i] != y_hat[i]):
                n_miss += 1

        print("Test complete\n")

        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("Total number of test samples = %d" %(m))
        print("Correctly classified         = %d" %(m -n_miss))
        print("Misclassified                = %d" %(n_miss))

        accuracy = float(m - n_miss) / float(m)
        print("Accuracy                     = %f%%" %(accuracy * 100))
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")




In [8]:
network = Network(features,classes, 6, [features,4,6,8,6,4,classes])
network.train(train_x,train_y)
network.test(test_x,test_y)

Iteration:  100  Error:  233.4726240294333
Iteration:  200  Error:  188.6909807702267
Iteration:  300  Error:  187.49038203515806
Iteration:  400  Error:  187.5135554055713
Iteration:  500  Error:  187.43621773486234
Iteration:  600  Error:  187.39960016405337
Iteration:  700  Error:  187.3916509198748
Iteration:  800  Error:  187.39027116196417
Iteration:  900  Error:  187.39004434808513
Iteration:  1000  Error:  187.3900072839383
Train done!
Expected:  4  Found:  3
Expected:  3  Found:  3
Expected:  2  Found:  3
Expected:  1  Found:  3
Expected:  4  Found:  3
Expected:  4  Found:  3
Expected:  4  Found:  3
Expected:  3  Found:  3
Expected:  1  Found:  3
Expected:  1  Found:  3
Expected:  1  Found:  3
Expected:  1  Found:  3
Expected:  3  Found:  3
Expected:  1  Found:  3
Expected:  3  Found:  3
Expected:  4  Found:  3
Expected:  3  Found:  3
Expected:  2  Found:  3
Expected:  4  Found:  3
Expected:  2  Found:  3
Expected:  2  Found:  3
Expected:  4  Found:  3
Expected:  3  Found:  3
